In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [ ]:
!pip install snowflake-connector-python


In [1]:
from snowflake.snowpark import Session

In [2]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [3]:
session = get_session()
# session.close()
session

connection_params =  {'user': 'PRAKHAR', 'password': 'Prak@apr24', 'account': 'ug94937.us-east4.gcp', 'warehouse': 'FOSFOR_INSIGHT_WH', 'database': 'test_database_prak', 'schema': 'test_schema_prak', 'role': 'PRAKHAR'}


In [4]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWSIGHT"

In [5]:
# from snowflake.ml.registry import model_registry
from snowflake.ml.registry import Registry as model_registry

In [6]:
!pip freeze | grep -i cloud

cloudpickle==3.0.0


In [ ]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [7]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [32]:
df = registry.show_models()
df
# type(df)

,created_on,name,database_name,schema_name,comment,owner,default_version_name,versions
0,2024-05-23 01:52:42.426000-07:00,MY_MODEL,MODEL_REGISTRY_SNOWSIGHT,SNOWSIGHT,None,PRAKHAR,V1,"[""V1"",""V2""]"


In [34]:
model_list = registry.models()
model_list

In [37]:
m =registry.get_model("my_model")
m

In [48]:
df_version = m.show_versions()
df_version

,created_on,name,comment,database_name,schema_name,module_name,is_default_version,functions,metadata,user_data
0,2024-05-23 01:52:42.486000-07:00,V1,My awesome ML model,MODEL_REGISTRY_SNOWSIGHT,SNOWSIGHT,MY_MODEL,false,"[""PREDICT""]",{},"{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."
1,2024-05-23 02:05:45.375000-07:00,V2,My awesome ML model,MODEL_REGISTRY_SNOWSIGHT,SNOWSIGHT,MY_MODEL,true,"[""PREDICT""]",{},"{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."


In [47]:
# default_version = m.default
m.default = "V2"

In [51]:
mv = m.default
mv

In [52]:
#Load default vesion as python object
clf = mv.load()

AttributeError: 'ModelVersion' object has no attribute 'load'

In [56]:
data_list = [ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9,
        10,  79,  82]
data_array = np.asarray(data_list)

remote_prediction = mv.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction.tolist()

ValueError: (2110) Data does not have the same number of features as signature. Signature requires 16 features, but have 1 in input data.

In [15]:
# import cloudpickle

# model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
# model_obj


array(['age', 'height_cm', 'weight_kg', 'skill_dribbling',
       'attacking_crossing', 'attacking_finishing',
       'movement_acceleration', 'movement_sprint_speed',
       'power_shot_power', 'mentality_aggression', 'defending_marking',
       'defending_standing_tackle', 'goalkeeping_diving',
       'goalkeeping_handling', 'overall', 'potential'], dtype=object)

In [26]:
type(x_train)

pandas.core.frame.DataFrame

In [20]:
# x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
# x_train
# type(x_train)

pandas.core.frame.DataFrame

In [31]:
mv = registry.log_model(gbm_default,
                   model_name="my_model",
                   version_name="v2",
                   conda_dependencies=["scikit-learn", "cloudpickle"],
                   comment="My awesome ML model",
#                     python_version="3.9",
#                    metrics={"score": 96},
                   sample_input_data=x_train)

/opt/conda/lib/python3.9/contextlib.py:119: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)
/tmp/pip_packages/snowflake/ml/model/model_signature.py:69: UserWarning: The sample input has 3750 rows, thus a truncation happened before inferring signature. This might cause inaccurate signature inference. If that happens, consider specifying signature manually.
  warnings.warn(


In [22]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [23]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

,short_name,nationality,overall,potential,wage_eur,value_eur,age,height_cm,weight_kg,attacking_crossing,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,Argentina,94,94,565000,95500000,32,170,72,88,...,75,96,33,37,26,6,11,15,14,8
1,Cristiano Ronaldo,Portugal,93,93,405000,58500000,34,187,83,84,...,85,95,28,32,24,7,11,15,14,11
2,Neymar Jr,Brazil,92,92,290000,105500000,27,175,68,87,...,90,94,27,26,29,9,9,15,15,11
3,J. Oblak,Slovenia,91,93,125000,77500000,26,188,87,13,...,11,68,27,12,18,87,92,78,90,89
4,E. Hazard,Belgium,91,91,470000,90000000,28,175,74,81,...,88,91,34,27,22,11,12,6,8,8
5,K. De Bruyne,Belgium,91,91,370000,90000000,28,181,70,93,...,79,91,68,58,51,15,13,5,10,13
6,M. ter Stegen,Germany,90,93,250000,67500000,27,187,85,18,...,25,70,25,13,10,88,85,88,88,90
7,V. van Dijk,Netherlands,90,91,200000,78000000,27,193,92,53,...,62,89,91,92,85,13,10,13,11,11
8,L. Modric,Croatia,90,90,340000,45000000,33,172,66,86,...,82,92,68,76,71,13,9,7,14,9
9,M. Salah,Egypt,90,90,240000,80500000,27,175,71,79,...,77,91,38,43,41,14,14,9,11,14


In [24]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

In [25]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

GradientBoostingRegressor()